In [16]:

import cv2
import tensorflow as tf
from tensorflow.keras import layers
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
import time
%matplotlib inline


In [4]:

ds = tfds.load('RockPaperScissors', split='train', as_supervised=True, shuffle_files=True)
 
ds = ds.shuffle(1000).batch(128)
 
@tf.function
def normalization(tensor):
    tensor = tf.image.resize(
    tensor, (128,128))
    tensor = tf.subtract(tf.divide(tensor, 127.5), 1)
    return tensor


2025-02-10 18:39:58.275245: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /home/dennis00/tensorflow_datasets/rock_paper_scissors/incomplete.UM824Y_3.0.0/rock_paper_scissors-t…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /home/dennis00/tensorflow_datasets/rock_paper_scissors/incomplete.UM824Y_3.0.0/rock_paper_scissors-t…

Dataset rock_paper_scissors downloaded and prepared to /home/dennis00/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


I0000 00:00:1739241607.279793  199334 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19917 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0


In [5]:

# label input
con_label = layers.Input(shape=(1,))
 
# latent vector input
latent_vector = layers.Input(shape=(100,))


In [6]:

def label_conditioned_generator(n_classes=3, embedding_dim=100):
    # embedding for categorical input
    label_embedding = layers.Embedding(n_classes, embedding_dim)(con_label)
    #print(label_embedding)
    # linear multiplication
    nodes = 4 * 4
    label_dense = layers.Dense(nodes)(label_embedding)
    # reshape to additional channel
    label_reshape_layer = layers.Reshape((4, 4, 1))(label_dense)
    return label_reshape_layer
 
def latent_input(latent_dim=100):
    # image generator input
    nodes = 512 * 4 * 4
    latent_dense = layers.Dense(nodes)(latent_vector)
    latent_dense = layers.ReLU()(latent_dense)
    latent_reshape = layers.Reshape((4, 4, 512))(latent_dense)
    return latent_reshape
    

In [7]:

def define_generator():
    latent_vector_output = label_conditioned_generator()
    label_output = latent_input()
    # merge label_conditioned_generator and latent_input output
    merge = layers.Concatenate()([latent_vector_output, label_output])
     
    x = layers.Conv2DTranspose(64 * 8, kernel_size=4, strides= 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_1')(merge)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_1')(x)
    x = layers.ReLU(name='relu_1')(x)
     
    x = layers.Conv2DTranspose(64 * 4, kernel_size=4, strides= 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_2')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_2')(x)
    x = layers.ReLU(name='relu_2')(x)
     
    x = layers.Conv2DTranspose(64 * 2, 4, 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_3')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8,  center=1.0, scale=0.02, name='bn_3')(x)
    x = layers.ReLU(name='relu_3')(x)
   
 
    x = layers.Conv2DTranspose(64 * 1, 4, 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_4')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8,  center=1.0, scale=0.02, name='bn_4')(x)
    x = layers.ReLU(name='relu_4')(x) 
     
     
    out_layer = layers.Conv2DTranspose(3, 4, 2,padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, activation='tanh', name='conv_transpose_6')(x)
     
     
   # define model
    model = tf.keras.Model([con_label, latent_vector], out_layer)
    return model

In [8]:

# label input
con_label = layers.Input(shape=(1,))
 
# input image
inp_img = layers.Input(shape=(128,128,3))


In [9]:

def label_condition_disc(in_shape=(128,128,3), n_classes=3, embedding_dim=100):
    # label input
    con_label = layers.Input(shape=(1,))
    # embedding for categorical input
    label_embedding = layers.Embedding(n_classes, embedding_dim)(con_label)
    nodes = in_shape[0] * in_shape[1] * in_shape[2]
    # scale up to image dimensions with linear layer
    label_dense = layers.Dense(nodes)(label_embedding)
    # reshape to a tensor
    label_reshape_layer = layers.Reshape((in_shape[0], in_shape[1], 3))(li)
    # image input
    return label_reshape_layer
 
 
def image_disc(in_shape=(128,128,3)):
    inp_image = layers.Input(shape=in_shape)
    return inp_image
    

In [10]:

def define_discriminator():
    label_condition_output = label_condition_disc()
    inp_image_output = image_disc()
    # concat label as a channel
    merge = layers.Concatenate()([inp_image_output, label_condition_output])
     
    x = layers.Conv2D(64, kernel_size=4, strides= 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_1')(merge)
    x = layers.LeakyReLU(0.2, name='leaky_relu_1')(x)
     
    x = layers.Conv2D(64 * 2, kernel_size=4, strides= 3, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_2')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_1')(x)
    x = layers.LeakyReLU(0.2, name='leaky_relu_2')(x)
     
    x = layers.Conv2D(64 * 4, 4, 3, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_3')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_2')(x)
    x = layers.LeakyReLU(0.2, name='leaky_relu_3')(x)
       
    x = layers.Conv2D(64 * 8, 4, 3,padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_5')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_4')(x)
    x = layers.LeakyReLU(0.2, name='leaky_relu_5')(x)
   
  
    flattened_out = layers.Flatten()(x)
    # dropout
    dropout = layers.Dropout(0.4)(flattened_out)
    # output
    dense_out = layers.Dense(1, activation='sigmoid')(dropout)
    # define model
 
 
    # define model
    model = tf.keras.Model([inp_image_output, con_label], dense_out)
    return model
    

In [11]:

binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()


In [13]:

def generator_loss(label, fake_output):
    gen_loss = binary_cross_entropy(label, fake_output)
    #print(gen_loss)
    return gen_loss


def discriminator_loss(label, output):
    disc_loss = binary_cross_entropy(label, output)
    #print(total_loss)
    return disc_loss
    

In [14]:

# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images,target):
    # noise vector sampled from normal distribution
    noise = tf.random.normal([target.shape[0], latent_dim])
    # Train Discriminator with real labels
    with tf.GradientTape() as disc_tape1:
        generated_images = conditional_gen([noise,target], training=True)
 
         
        real_output = conditional_discriminator([images,target], training=True)
        real_targets = tf.ones_like(real_output)
        disc_loss1 = discriminator_loss(real_targets, real_output)
         
    # gradient calculation for discriminator for real labels    
    gradients_of_disc1 = disc_tape1.gradient(disc_loss1, conditional_discriminator.trainable_variables)
     
    # parameters optimization for discriminator for real labels   
    discriminator_optimizer.apply_gradients(zip(gradients_of_disc1,\
    conditional_discriminator.trainable_variables))
     
    # Train Discriminator with fake labels
    with tf.GradientTape() as disc_tape2:
        fake_output = conditional_discriminator([generated_images,target], training=True)
        fake_targets = tf.zeros_like(fake_output)
        disc_loss2 = discriminator_loss(fake_targets, fake_output)
    # gradient calculation for discriminator for fake labels 
    gradients_of_disc2 = disc_tape2.gradient(disc_loss2, conditional_discriminator.trainable_variables)
     
     
    # parameters optimization for discriminator for fake labels        
    discriminator_optimizer.apply_gradients(zip(gradients_of_disc2,\
    conditional_discriminator.trainable_variables))
     
    # Train Generator with real labels
    with tf.GradientTape() as gen_tape:
        generated_images = conditional_gen([noise,target], training=True)
        fake_output = conditional_discriminator([generated_images,target], training=True)
        real_targets = tf.ones_like(fake_output)
        gen_loss = generator_loss(real_targets, fake_output)
 
    # gradient calculation for generator for real labels     
    gradients_of_gen = gen_tape.gradient(gen_loss, conditional_gen.trainable_variables)
     
    # parameters optimization for generator for real labels
    generator_optimizer.apply_gradients(zip(gradients_of_gen,\
    conditional_gen.trainable_variables)) 
    

In [37]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()
    for image_batch, target in dataset:
      img = tf.cast(image_batch, tf.float32)
      imgs = normalization(img)
      train_step(imgs, target)
 
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
 
train(ds, 200)

StagingError: in user code:

    File "/tmp/ipykernel_199334/3641390538.py", line 6, in train_step  *
        noise = tf.random.normal([target.shape[0], latent_dim])

    IndexError: tuple index out of range


In [49]:
ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [48]:
ds = tfds.load('RockPaperScissors', split='train', as_supervised=True, shuffle_files=True)

for image_batch, target in ds:
    print(image_batch.shape)
    print(target.numpy())

(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
2
(300, 300, 3)
2
(300, 300, 3)
2
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
2
(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
1
(300, 300, 3)
2
(300, 300, 3)
1
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
1
(300, 300, 3)
2
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
2
(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
2
(300, 300, 3)
2
(300, 300, 3)
1
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
0
(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
1
(300, 300, 3)
2
(300, 300, 3)
0
(300, 300, 3)
1
(300, 300, 3)
0
(300, 300, 3)
1
(300, 300, 3)
2
(300, 300, 3)
1
(300, 300, 3)
0
(300, 300, 3)
1
(300, 300, 3)
2
(300, 30

In [31]:

def preprocess(image, label):
    image = tf.cast(image, tf.uint8)
    return image, label

ds = ds.map(preprocess)


In [32]:
ds

<_MapDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [26]:

def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()
    for image_batch, _ in dataset:
      img = tf.cast(image_batch, tf.int32)
      imgs = normalization(img)
      train_step(imgs, target)
 
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
 
train(ds, 200)


NameError: name 'target' is not defined

In [23]:
ds

<_MapDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [25]:
ds

<_MapDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>